In [158]:
#limpa df, comeca etc

import pandas as pd
import numpy as np
import re

df = pd.read_csv('data/IMDB.csv' , encoding='utf-8')

def clean_t(t):
    t_clean = re.sub(r'<.*?>', '', t)
    t_clean = re.sub(r'[^\w\s]', '', t_clean)
    return t_clean

df['review'] = df['review'].apply(clean_t)
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production The filming tech...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically theres a family where a little boy J...,negative
4,Petter Matteis Love in the Time of Money is a ...,positive


In [159]:
#fazendo o treino
df_treino = df.iloc[:15000]
df_treino.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production The filming tech...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically theres a family where a little boy J...,negative
4,Petter Matteis Love in the Time of Money is a ...,positive


In [160]:
#fazendo a dict de palavras nao repetidas
# primeiro bullet point
df_positivo = df_treino[df_treino['sentiment'] == 'positive']
df_negativo = df_treino[df_treino['sentiment'] == 'negative']
numero_t_positivo = len(df_positivo)
numero_t_negativo = len(df_negativo)

# Inicializa os dicionários de contagem para positivos e negativos
contagem_palavras = {}

# Itera sobre os textos e sentimentos
for texto, sentimento in zip(df_treino['review'], df_treino['sentiment']):
    palavras = set(texto.lower().split())  # Divide o texto em palavras
    for palavra in palavras:
        if palavra not in contagem_palavras:
            contagem_palavras[palavra] = {'positive': 0, 'negative': 0}
        contagem_palavras[palavra][sentimento] += 1


prob_palavra_dado_classe = {}

for palavra2,dictt in contagem_palavras.items():
    pos = (dictt['positive']) / (numero_t_positivo)
    nega = (dictt['negative']) / (numero_t_negativo)
    prob_palavra_dado_classe[palavra2] = {'positive': pos, 'negative': nega}




prob_not_palavra_dado_classe = {}

for palavra3,dictt2 in prob_palavra_dado_classe.items():
    pos2 = 1 - dictt2['positive']
    nega2 = 1 - dictt2['negative']
    prob_not_palavra_dado_classe[palavra3] = {'positive': pos2, 'negative': nega2}



In [161]:
#estimando P(palavra)
#segundo bullet point

total_textos = len(df_treino)

prob_palavra = {}

for palavra,contagem in contagem_palavras.items():
    prob_palavra[palavra] = (contagem['positive'] + contagem['negative']) / total_textos



In [162]:
#calculando P(classe|texto)
from math import log10
lista = []
a = []

for texto in df_treino['review']:
    p_pos = log10(0.5) 
    p_neg = log10(0.5)
    texto = set(texto.lower().split())
    for palavra in texto:
        if palavra in prob_not_palavra_dado_classe:
            p_pos += log10(prob_palavra_dado_classe[palavra]['positive']+0.001)
            p_neg += log10(prob_palavra_dado_classe[palavra]['negative']+0.001)
    for not_palavra in prob_not_palavra_dado_classe:
        if not_palavra not in texto:
            if not_palavra in prob_not_palavra_dado_classe:
                p_pos += log10(prob_not_palavra_dado_classe[not_palavra]['positive']+0.001)
                p_neg += log10(prob_not_palavra_dado_classe[not_palavra]['negative']+0.001)
    lista.append(p_pos)
    lista.append(p_neg)


In [163]:
dic_review_classe = {}

    
pos = 0
contador = 1
for chance_pos in lista:
    pos += 1
    texto = f"Texto{contador}"
    if (pos % 2 == 1 or pos == 1) and pos != len(lista)-1:
        if chance_pos > lista[pos]:
            contador += 1
            dic_review_classe[texto] = 'positive'
        else:
            contador += 1
            dic_review_classe[texto] = 'negative'

In [164]:
lista_sentimento_certa = []
for sentimento in df_treino['sentiment']:
    lista_sentimento_certa.append(sentimento)

In [165]:
#calculando acuracia
acuracia = 0
for sentimento_certo, sentimento_estimado in zip(lista_sentimento_certa, dic_review_classe.values()):
    if sentimento_certo == sentimento_estimado:
        acuracia += 1
acuracia = acuracia / len(lista_sentimento_certa)
print(acuracia)

0.9024


In [166]:
df_teste = df.iloc[15000:]
df_teste.head(10)

,review,sentiment
15000,This movie really shows its age The print I sa...,negative
15001,The title should have been the walker The guy ...,negative
15002,Sequels are a capricious lot with most nowhere...,positive
15003,I really wish Hollywood would come up with som...,negative
15004,I wasnt quite sure if this was just going to b...,positive
15005,This movie came to me highly recommended by Ma...,positive
15006,Warning spoilers although its hard to spoil t...,positive
15007,I am not understanding why people are praising...,negative
15008,Due to this show getting cut early I never rea...,positive
15009,If youve ever had a mad weekend out with your ...,positive


In [167]:
#Avalia todos os textos da df teste
dic_review_teste = {}

for texto in df_teste['review']:
    p_pos = log10(0.5) 
    p_neg = log10(0.5)
    texto = set(texto.lower().split())
    for palavra in texto:
        if palavra in prob_not_palavra_dado_classe:
            p_pos += log10(prob_palavra_dado_classe[palavra]['positive']+0.001)
            p_neg += log10(prob_palavra_dado_classe[palavra]['negative']+0.001)
    for not_palavra in prob_not_palavra_dado_classe:
        if not_palavra not in texto:
            if not_palavra in prob_not_palavra_dado_classe:
                p_pos += log10(prob_not_palavra_dado_classe[not_palavra]['positive']+0.001)
                p_neg += log10(prob_not_palavra_dado_classe[not_palavra]['negative']+0.001)
    lista.append(p_pos)
    lista.append(p_neg)

    
pos = 0
contador = 1
for chance_pos in lista:
    pos += 1
    texto = f"Texto{contador}"
    if (pos % 2 == 1 or pos == 1) and pos != len(lista)-1:
        if chance_pos > lista[pos]:
            contador += 1
            dic_review_teste[texto] = 'positive'
        else:
            contador += 1
            dic_review_teste[texto] = 'negative'

print(dic_review_teste)


{'Texto1': 'negative', 'Texto2': 'positive', 'Texto3': 'positive', 'Texto4': 'negative', 'Texto5': 'positive', 'Texto6': 'positive', 'Texto7': 'positive', 'Texto8': 'negative', 'Texto9': 'negative', 'Texto10': 'positive', 'Texto11': 'negative', 'Texto12': 'negative', 'Texto13': 'negative', 'Texto14': 'negative', 'Texto15': 'positive', 'Texto16': 'negative', 'Texto17': 'positive', 'Texto18': 'negative', 'Texto19': 'positive', 'Texto20': 'negative', 'Texto21': 'negative', 'Texto22': 'negative', 'Texto23': 'positive', 'Texto24': 'negative', 'Texto25': 'negative', 'Texto26': 'positive', 'Texto27': 'negative', 'Texto28': 'negative', 'Texto29': 'negative', 'Texto30': 'positive', 'Texto31': 'positive', 'Texto32': 'positive', 'Texto33': 'negative', 'Texto34': 'positive', 'Texto35': 'negative', 'Texto36': 'negative', 'Texto37': 'negative', 'Texto38': 'negative', 'Texto39': 'positive', 'Texto40': 'negative', 'Texto41': 'negative', 'Texto42': 'positive', 'Texto43': 'negative', 'Texto44': 'negativ

In [168]:
#calculando precisao
lista_sentimento_certa = []
for sentimento in df_teste['sentiment']:
    lista_sentimento_certa.append(sentimento)

acuracia = 0
for sentimento_certo, sentimento_estimado in zip(lista_sentimento_certa, dic_review_teste.values()):
    if sentimento_certo == sentimento_estimado:
        acuracia += 1
acuracia = acuracia / len(lista_sentimento_certa)
print(acuracia)

0.4947714285714286
